In [ ]:
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Activation, Dropout, Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import pandas as pd
np.random.seed(25)

In [ ]:
#Load and split databases
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
# Show some images in the dataset
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
import matplotlib.pyplot as plt
plt.figure(figsize=(8,10))
for i in range(64):
    plt.subplot(8,8,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])

In [ ]:
plt.imshow(X_train[1005], cmap='gray')
plt.title('Class '+ str(y_train[1005]))
plt.show()

In [ ]:
import seaborn as sns
sns.countplot(y_train)
plt.show()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0

In [ ]:
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

In [ ]:
batch_size = 128
num_epoch = 50
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

In [ ]:
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="save/fashion_cnn1.hdf5", verbose=0, save_best_only=True) # save best model
learning_rate = 0.001

model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.25))
model1.add(Dense(number_of_classes, activation='softmax'))

monitor = EarlyStopping(monitor='val_accuracy', min_delta=1e-3, patience=5, verbose=1, mode='auto')

model1.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learning_rate), metrics=['accuracy'])

model1.summary()

In [ ]:
# Plot Model
from keras.utils.vis_utils import plot_model
plot_model(model1, to_file='cnn_model1.png', show_layer_names=False)

In [ ]:
import time
beg = time.time()

callbacks=[monitor,checkpointer],

history = model1.fit(X_train,Y_train, batch_size=128, epochs=50, verbose=2, callbacks=[monitor], validation_data=(X_test, Y_test), shuffle=True) 

end = time.time()
cnn1_mnist_time = end-beg
print("Total Train Time: {:0.2f} minute".format(cnn1_mnist_time/60.0))

In [ ]:
score = model1.evaluate(X_test, Y_test)
print('Testing Loss:', score[0])
print('Testing Accuracy: ', score[1])
score = model1.evaluate(X_train, Y_train)
print('Train Loss:', score[0])
print('Train Accuracy: ', score[1])

In [ ]:
from sklearn.metrics import classification_report
y_hat = model1.predict(X_test)
y_pred = np.argmax(y_hat, axis=1)
y_true = np.argmax(Y_test, axis=1)
print(classification_report(y_true, y_pred))

In [ ]:
import os
# plotting the metrics
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Test'], loc='lower right')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Test'], loc='upper right')
plt.tight_layout()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
Y_pred = model1.predict(X_test, verbose=2)
y_pred = np.argmax(Y_pred, axis=1)
 
for ix in range(10):
    print(ix, confusion_matrix(np.argmax(Y_test,axis=1),y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(Y_test,axis=1),y_pred)
print(cm)
 
# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd
 
 
df_cm = pd.DataFrame(cm, range(10),
                  range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()